In [1]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass


In [4]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "A.N.Other@example.com"
import pandas as pd
import xml.etree.ElementTree as ET

In [6]:
dis = pd.read_table('./curated_gene_disease_associations.tsv')
dis

,geneId,geneSymbol,diseaseId,diseaseName,score,NofPmids,NofSnps,source
0,10,NAT2,C0005695,Bladder Neoplasm,0.245871,5,0,CTD_human
1,10,NAT2,C0013182,Drug Allergy,0.202682,3,0,CTD_human
2,10,NAT2,C0023530,Leukopenia,0.200000,1,0,CTD_human
3,10,NAT2,C0023903,Liver neoplasms,0.209628,1,0,CTD_human
4,10,NAT2,C0027794,Neural Tube Defects,0.200275,1,0,CTD_human
...,...,...,...,...,...,...,...,...
130816,9997,SCO2,C2750915,Gliosis in the basal ganglia,0.200000,0,0,HPO
130817,9997,SCO2,C2752013,Prenatal onset,0.200000,0,0,HPO
130818,9997,SCO2,C4020875,Mental and motor retardation,0.200000,0,0,HPO
130819,9997,SCO2,C4020899,Autosomal recessive predisposition,0.200000,0,0,HPO


In [9]:
schis_db = dis.loc[dis['diseaseName'] == 'Prenatal onset']
schis_db

,geneId,geneSymbol,diseaseId,diseaseName,score,NofPmids,NofSnps,source
621,10021,HCN4,C2752013,Prenatal onset,0.2,0,0,HPO
1953,1020,CDK5,C2752013,Prenatal onset,0.2,0,0,HPO
3456,10381,TUBB3,C2752013,Prenatal onset,0.2,0,0,HPO
3548,1041,CDSN,C2752013,Prenatal onset,0.2,0,0,HPO
4190,10518,CIB2,C2752013,Prenatal onset,0.2,0,0,HPO
...,...,...,...,...,...,...,...,...
125409,925,CD8A,C2752013,Prenatal onset,0.2,0,0,HPO
127498,9526,MPDU1,C2752013,Prenatal onset,0.2,0,0,HPO
128426,9662,CEP135,C2752013,Prenatal onset,0.2,0,0,HPO
129511,9841,ZBTB24,C2752013,Prenatal onset,0.2,0,0,HPO


In [11]:
handle = Entrez.efetch(db="gene", id="10021", rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print(start, end, gi_id, strand)

73319859 73368958 568815583 minus


In [17]:
#Выгрузить с NCBI нуклеотидную последовательность X хромосомы, для этого узнать ID хромосомы - NC_000015.10
handle = Entrez.efetch(db="nucleotide",
                       id="NC_000015.10",
                       rettype = 'fasta',
                       retmode = 'text')
#загружаем
chromosome = SeqIO.read(handle, 'fasta')

In [18]:
#а та ли это хромосома?
chromosome.description

'NC_000015.10 Homo sapiens chromosome 15, GRCh38.p14 Primary Assembly'

In [19]:
#вырезаем из хромосомы ген, для этого узнаем локацию нужного гена. Для HCN4 это 73319859..73368958
geneHCN4 = chromosome.seq[73319859:73368958]
print(geneHCN4)
len(geneHCN4)

...


49099